In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import mean_squared_error,mean_absolute_error

In [3]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(Dataset_path+"ml-latest/ratings.csv")

### just keep rating year between 1995, 2001

In [4]:
from datetime import datetime

# strftime('%Y-%m-%d %H:%M:%S')
ratings['year_rated'] = ratings['timestamp'].apply(lambda x: int( datetime.fromtimestamp( x ).strftime('%Y') ) )

ratings = ratings[ ratings['year_rated'].between(1995, 2001)].copy()

ratings.reset_index(drop=True, inplace=True)

In [5]:
print( f"number of Ratings : { ratings.shape[0] }")
print( f"number of movies : { ratings.groupby('movieId').count().shape[0] }")
print( f"number of users : { ratings.groupby('userId').count().shape[0] }")
print( f"range of rating : ( { np.min( ratings.groupby('rating').count().index )}, {np.max( ratings.groupby('rating').count().index )})  ")

number of Ratings : 7329482
number of movies : 4937
number of users : 103827
range of rating : ( 1.0, 5.0)  


In [15]:
MyCol = ['movieId','year','duration','metascore','reviews_from_users','reviews_from_critics'
         ,'country','language','director','production_company','mean_vote']
FinalDataset = pd.merge(ratings, Movies_metadata[ MyCol ], how='inner')

In [16]:
FinalDataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7206877 entries, 0 to 7206876
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   userId                int64  
 1   movieId               int64  
 2   rating                float64
 3   timestamp             int64  
 4   year_rated            int64  
 5   year                  int64  
 6   duration              int64  
 7   metascore             float64
 8   reviews_from_users    float64
 9   reviews_from_critics  float64
 10  country               object 
 11  language              object 
 12  director              object 
 13  production_company    object 
 14  mean_vote             float64
dtypes: float64(5), int64(6), object(4)
memory usage: 879.7+ MB


In [17]:
myobj = ['country','language','director','production_company']
obj_FinalDataset= FinalDataset.select_dtypes(include=['object']).copy()

for i in myobj:
    obj_FinalDataset[i] = obj_FinalDataset[i].astype('category')
    FinalDataset[i] = obj_FinalDataset[i].cat.codes
    
FinalDataset = FinalDataset.fillna(100000)
FinalDataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7206877 entries, 0 to 7206876
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   userId                int64  
 1   movieId               int64  
 2   rating                float64
 3   timestamp             int64  
 4   year_rated            int64  
 5   year                  int64  
 6   duration              int64  
 7   metascore             float64
 8   reviews_from_users    float64
 9   reviews_from_critics  float64
 10  country               int16  
 11  language              int16  
 12  director              int16  
 13  production_company    int16  
 14  mean_vote             float64
dtypes: float64(5), int16(4), int64(6)
memory usage: 714.8 MB


In [19]:
data =  FinalDataset[['userId' ,'movieId','year','duration','metascore','reviews_from_users','reviews_from_critics'
                              ,'country','language','director','production_company','mean_vote','rating']]

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(data.drop(['movieId','userId','rating'],axis=1))
data_norm = pd.DataFrame(scaled,columns=(data.drop(['movieId','userId','rating'],axis=1).columns))
data_norm= data_norm.join(data['userId'])
data_norm= data_norm.join(data['movieId'])
data_norm= data_norm.join(data['rating'])

                               
from scipy import stats
z_scores = stats.zscore(data_norm.drop(['movieId','mean_vote'],axis=1))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
newdata = data_norm[filtered_entries]

newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6578912 entries, 0 to 7206874
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   year                  float64
 1   duration              float64
 2   metascore             float64
 3   reviews_from_users    float64
 4   reviews_from_critics  float64
 5   country               float64
 6   language              float64
 7   director              float64
 8   production_company    float64
 9   mean_vote             float64
 10  userId                int64  
 11  movieId               int64  
 12  rating                float64
dtypes: float64(11), int64(2)
memory usage: 702.7 MB


In [36]:
target = newdata['rating'] 
predictors = newdata.drop(['userId','movieId','rating'],axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)

In [53]:
model01 = Sequential()
model01.add(Dense(200, activation='relu', input_dim=10))
model01.add(Dense(100, activation='relu'))
model01.add(Dense(50, activation='relu'))
model01.add(Dense(25, activation='relu'))
model01.add(Dense(1, activation='linear'))

# Compile the model
model01.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [ ]:
model01.fit(X_train, y_train, epochs=5, batch_size=10)

Epoch 1/5
 76838/460524 [====>.........................] - ETA: 14:10 - loss: 0.7705 - mean_squared_error: 1.0340